<a href="https://colab.research.google.com/github/mimingucci/ML/blob/main/QuadraticDiscriminativeAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Similar to Linear Discriminative Analysis(LDA) but each class has idividual covariance matrix <br/>
$\widehat{\Sigma _{k}}=\frac{1}{N_{k}}\sum_{n\in C_{k}}^{}(x_{n}-\mu _{k})(x_{n}-\mu _{k})^{T}$ and others are same

Note:<br/>
PDF of MVN has formula : $(2\pi )^{-k/2}det(\Sigma )^{-1/2}exp(-\frac{1}{2}(x-\mu )^{T}\Sigma ^{-1}(x-\mu ))$ <br/>condition: $\mu \in R^{k} $ and $\Sigma \in R^{k * k}$

In [1]:
import numpy as np
class QDA:

    ## Fitting the model

    def fit(self, X, y):

        ## Record info
        self.N, self.D = X.shape
        self.X = X
        self.y = y


        ## Get prior probabilities
        self.unique_y, unique_y_counts = np.unique(self.y, return_counts = True) # returns unique y and counts
        self.pi_ks = unique_y_counts/self.N


        ## Get mu and Sigma for each class
        self.mu_ks = []
        self.Sigma_ks = []
        for i, k in enumerate(self.unique_y):

            X_k = self.X[self.y == k]
            mu_k = X_k.mean(0).reshape(self.D, 1)
            self.mu_ks.append(mu_k)

            Sigma_k = np.zeros((self.D, self.D))
            for x_n in X_k:
                x_n = x_n.reshape(-1,1)
                x_n_minus_mu_k = (x_n - mu_k)
                Sigma_k += np.dot(x_n_minus_mu_k, x_n_minus_mu_k.T)
            self.Sigma_ks.append(Sigma_k/len(X_k))

    ## Making classifications

    def _mvn_density(self, x_n, mu_k, Sigma_k):
        x_n_minus_mu_k = (x_n - mu_k)
        density = np.linalg.det(Sigma_k)**(-1/2) * np.exp(-(1/2)*x_n_minus_mu_k.T @ np.linalg.inv(Sigma_k) @ x_n_minus_mu_k)
        return density

    def classify(self, X_test):

        y_n = np.empty(len(X_test))
        for i, x_n in enumerate(X_test):

            x_n = x_n.reshape(-1, 1)
            p_ks = np.empty(len(self.unique_y))

            for j, k in enumerate(self.unique_y):

                p_x_given_y = self._mvn_density(x_n, self.mu_ks[j], self.Sigma_ks[j])
                p_y_given_x = self.pi_ks[j]*p_x_given_y
                p_ks[j] = p_y_given_x

            y_n[i] = self.unique_y[np.argmax(p_ks)]

        return y_n
